In [1]:
#!pip3 install kerastuner

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


from sklearn.datasets import make_blobs
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef,cohen_kappa_score,balanced_accuracy_score


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LeakyReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.regularizers import l2


from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling,margin_sampling,entropy_sampling
from modAL.disagreement import KL_max_disagreement

from functools import partial


import warnings

warnings.filterwarnings('ignore')

2023-05-26 03:53:55.997674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
float_id='PR_PF_4902337'

data_source="./dataset/PR_PF_4902337.csv"
data= pd.read_csv(data_source)
data

,Date,Latitude,longitude,Pressure,Salinity,Temperature,QC,Label
0,-1.585326,-1.728482,0.521565,-0.884740,1.263216,0.821065,0,0
1,-1.585326,-1.728482,0.521565,-0.883036,1.263216,0.820925,0,0
2,-1.585326,-1.728482,0.521565,-0.881261,1.262159,0.820925,0,0
3,-1.585326,-1.728482,0.521565,-0.879487,1.262159,0.821065,0,0
4,-1.585326,-1.728482,0.521565,-0.877641,1.262159,0.821065,0,0
...,...,...,...,...,...,...,...,...
214895,1.974276,-1.605324,-0.343149,-0.800825,1.282237,1.124021,0,0
214896,1.974276,-1.605324,-0.343149,-0.800612,1.282237,1.123881,0,0
214897,1.974276,-1.605324,-0.343149,-0.800328,1.281180,1.123741,0,0
214898,1.974276,-1.605324,-0.343149,-0.800186,1.283294,1.123601,0,0


In [4]:
row,col=data.shape

In [5]:
#def compute ratio
def comp_ratio(data):
    instance = data[(data['Label']==1)]
    rate=len(instance)/len(data)*100
    print(rate)
    return rate

In [6]:
dirty_ratio = comp_ratio(data)
dirty_ratio

0.08189855746859004


0.08189855746859004

# Splite Dataset

In [7]:
train_set=pd.read_csv('./randomtrain/'+float_id+'.csv')
test_set=pd.read_csv('./randomtest/'+float_id+'.csv')
len(train_set),len(test_set)

(150430, 64470)

In [8]:
#train_set2,test_set2=getLastSplit(data,0.3,dirty_ratio)

In [9]:
comp_ratio(train_set)
comp_ratio(test_set)

0.08176560526490727
0.08220877927718319


0.08220877927718319

In [10]:
#this function use random grid search to find best parameters
def getPar(model,dist,data,niter):
    from sklearn.model_selection import RandomizedSearchCV
    x,y=data.shape
    clf = model

    param_dist = dist
    grid = RandomizedSearchCV(clf,param_dist,cv = 3,scoring = "balanced_accuracy",n_iter=niter,n_jobs = -1)

    #train
    grid.fit(data.iloc[:,0:y-1],data.iloc[:,y-1])
    #get best parameter
    print(grid.best_score_)
    return grid.best_params_

In [11]:
# metric
def computeMetric(y_tru,y_pre):
    acc = accuracy_score(y_tru,y_pre)
    pre=precision_score(y_tru,y_pre)
    recall=recall_score(y_tru,y_pre)
    cm=confusion_matrix(y_tru,y_pre)
    f1 = f1_score(y_tru,y_pre)
    mcc=matthews_corrcoef(y_tru, y_pre)
    kappa=cohen_kappa_score(y_tru, y_pre)
    bac=balanced_accuracy_score(y_tru,y_pre)
    print("acc:",acc)
    print("balanced acc:",bac)
    print("precision:",pre)
    print("recall:",recall)
    print("cm:",cm)
    print("f1:",f1)
    print("MCC:", mcc)
    print("Kappa:",kappa)
    
#     confusion matrix
#     cmap1 = sns.diverging_palette(260,-10,s=50, l=75, n=5, as_cmap=True)
#     plt.subplots(figsize=(12,8))
#     cf_matrix = confusion_matrix(y_tru, y_pre)
#     sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap = cmap1, annot = True, annot_kws = {'size':15})
    
    return kappa

# splite train(pool), test, Labeled, Unlabeled

In [32]:
x,y=train_set.shape
#pool 
X_Pool = train_set.iloc[:,0:y-2].values
y_Pool = train_set.iloc[:,y-1].values

In [33]:
#test set
X_test=test_set.iloc[:,0:y-2]
y_true=test_set.iloc[:,y-1]
print(len(X_test),len(y_true))

64470 64470


In [34]:
X_test

,Date,Latitude,longitude,Pressure,Salinity,Temperature
0,1.026600,1.087160,0.035116,-0.504920,-0.316596,-0.574886
1,1.072028,1.112070,0.030377,-0.135678,-0.431780,-0.923664
2,0.846123,1.307278,0.285852,-0.160739,-0.173938,-0.444848
3,0.372511,1.697205,0.684481,-0.813320,-3.217321,-1.273422
4,0.282289,1.655550,0.492615,-0.224492,-0.294405,-0.583434
...,...,...,...,...,...,...
64465,-1.147309,-1.612102,-0.744355,-0.710733,1.174450,0.884823
64466,0.034099,1.016379,0.733566,0.119760,-0.341958,-0.828797
64467,1.613443,-0.329653,-2.164174,-0.828938,0.214939,0.948581
64468,-1.400874,-1.288755,-0.224789,0.968214,-0.294405,-0.771485


In [35]:
# Set RNG seed for reproducibility.
RANDOM_STATE_SEED = 123
np.random.seed(RANDOM_STATE_SEED)

n_initial=100
N_QUERIES = 200

# Save initial training data

In [36]:
# X_initial,y_initial,X_re,y_re=initial_data(n_initial,X_Pool,y_Pool)
# df_X_i = pd.DataFrame(X_initial)
# df_y_i = pd.DataFrame(y_initial)
# df_X_r = pd.DataFrame(X_re)
# df_y_r = pd.DataFrame(y_re)


X_initial=pd.read_csv('./randomtrain/'+float_id+'_X_initial.csv').values
y_initial=pd.read_csv('./randomtrain/'+float_id+'_y_initial.csv').values.ravel()
X_re=pd.read_csv('./randomtrain/'+float_id+'_X_re.csv').values
y_re=pd.read_csv('./randomtrain/'+float_id+'_y_re.csv').values.ravel()
print(X_re.shape,X_initial.shape)
print(y_re.shape,y_initial.shape)

(149230, 6) (1200, 6)
(149230,) (1200,)


# molAL

In [37]:
def random_sampling(classifier, X_pool):
    n_samples = len(X_pool)
    query_idx = np.random.choice(range(n_samples))
    return query_idx, X_pool[query_idx]

In [38]:
def al(clf,strategy,X_L,y_L):
    learner = ActiveLearner(estimator=clf,
                            query_strategy=strategy,
                            X_training=X_L, y_training=y_L)
    return learner

In [39]:
def al_learn(model,sampling,X_re,y_re,X_t):
    X_L = X_initial.copy()
    y_L = y_initial.copy()
    X_U,y_U =X_re.copy(),y_re.copy()
    y_pre=model.predict(X_t)
    y_pre = (y_pre > 0.5).astype(int)
    unqueried_kappa=cohen_kappa_score(y_true, y_pre)
    unqueried_f1=f1_score(y_true,y_pre)
    print(unqueried_kappa,unqueried_f1)
    kappa_history = [unqueried_kappa]
    f1_history = [unqueried_f1]
    # the active learning loop
    n_queries = 40
    for idx in range(n_queries):
        query_idx = np.random.choice(range(len(y_U)),size=5)
        print(query_idx)
        X=X_U[query_idx]
        y=y_U[query_idx]
        print(idx+1,"query label --------------------->",y_re[query_idx])
        # update model on new data only with a smaller learning rate
        optimizer = RMSprop(learning_rate=0.0001)
        # compile the model
        model.compile(optimizer=optimizer, loss='binary_crossentropy')
        # fit the model on new data
        model.fit(X, y, epochs=100, verbose=0)
        # remove queried instance from pool
        X_U = np.delete(X_U, query_idx, axis=0)
        y_U = np.delete(y_U, query_idx, axis=0)
        y_pre=model.predict(X_t)
        y_pre = (y_pre > 0.5).astype(int)
        kappa=cohen_kappa_score(y_true, y_pre)
        f1=f1_score(y_true,y_pre)
        print(idx+1,"-------------------->",kappa,f1)
        # Recall precision F1
        kappa_history.append(kappa)
        f1_history.append(f1)
    #print(len(X_U))
    df_scores= pd.concat([pd.DataFrame(kappa_history,columns=['kappa']), 
                          pd.DataFrame(f1_history,columns=['f1'])],
                         axis=1)
    return df_scores


In [40]:
# model = Sequential()
# model.add(Dense(units=64, kernel_initializer='normal', activation='sigmoid', input_dim=X_Pool.shape[1]))
# model.add(Dropout(0.2))
# model.add(Dense(units=1, kernel_initializer='normal', activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])
# history = model.fit(X_Pool, y_Pool, batch_size=64, epochs=50, validation_data=(X_test, y_true))

In [41]:
def f1(y_true, y_pred):
    y_pred = tf.round(y_pred)
    return f1_score(y_true, y_pred)

In [42]:
# build function for the Keras' scikit-learn API
def create_baseline_model():
    model_b = Sequential()
    model_b.add(Dense(units=128, kernel_initializer='he_normal', input_dim=X_initial.shape[1]))
    model_b.add(LeakyReLU())
    model_b.add(Dropout(0.5))
    model_b.add(Dense(units=64, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)))
    model_b.add(LeakyReLU())
    model_b.add(Dropout(0.3))
    model_b.add(Dense(units=32, kernel_initializer='he_normal', kernel_regularizer=l2(0.001)))
    model_b.add(LeakyReLU())
    model_b.add(Dense(units=1, kernel_initializer='he_normal', activation='sigmoid'))
    optimizer = RMSprop(learning_rate=0.001)
    model_b.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['binary_crossentropy'])

    return model_b


# create the classifier
model_b = create_baseline_model()

In [43]:
print(X_initial.shape, y_initial.shape, X_test.shape, y_true.shape)

(1200, 6) (1200,) (64470, 6) (64470,)


In [44]:
history = model_b.fit(X_initial, y_initial, batch_size=32, epochs=50, validation_data=(X_test, y_true))

Epoch 1/50
38/38 [==============================] - 2s 31ms/step - loss: 0.4301 - binary_crossentropy: 0.2421 - val_loss: 0.1950 - val_binary_crossentropy: 0.0093
Epoch 2/50
38/38 [==============================] - 1s 28ms/step - loss: 0.1951 - binary_crossentropy: 0.0153 - val_loss: 0.1787 - val_binary_crossentropy: 0.0082
Epoch 3/50
38/38 [==============================] - 1s 28ms/step - loss: 0.1702 - binary_crossentropy: 0.0140 - val_loss: 0.1472 - val_binary_crossentropy: 0.0086
Epoch 4/50
38/38 [==============================] - 1s 28ms/step - loss: 0.1341 - binary_crossentropy: 0.0080 - val_loss: 0.1222 - val_binary_crossentropy: 0.0091
Epoch 5/50
38/38 [==============================] - 1s 28ms/step - loss: 0.1154 - binary_crossentropy: 0.0140 - val_loss: 0.0985 - val_binary_crossentropy: 0.0091
Epoch 6/50
38/38 [==============================] - 1s 28ms/step - loss: 0.0892 - binary_crossentropy: 0.0104 - val_loss: 0.0775 - val_binary_crossentropy: 0.0089
Epoch 7/50
38/38 [====

In [45]:
y_pre=model_b.predict(X_test)
y_pred = (y_pre > 0.5).astype(int)
computeMetric(y_true,y_pred)

2015/2015 [==============================] - 1s 548us/step
acc: 0.9991779122072282
balanced acc: 0.5
precision: 0.0
recall: 0.0
cm: [[64417     0]
 [   53     0]]
f1: 0.0
MCC: 0.0
Kappa: 0.0


0.0

# RNN

In [46]:
# Reshape the input data to be 3D
print(X_Pool.shape,X_test.shape)
X_Pool_3 = X_Pool.reshape(X_Pool.shape[0], 1, X_Pool.shape[1])
X_initial_3=X_initial.reshape(X_initial.shape[0], 1, X_initial.shape[1])
X_re_3=X_re.reshape(X_re.shape[0], 1,X_re.shape[1])
X_test_3 = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

(150430, 6) (64470, 6)


In [47]:
print(y_Pool.shape,y_true.shape)

(150430,) (64470,)


In [48]:
# Build the RNN binary classification model
def create_RNN_model():
    model_R = Sequential()
    model_R.add(SimpleRNN(units=32, activation='relu', input_shape=(1, X_Pool.shape[1])))
    model_R.add(Dropout(0.2))
    model_R.add(Dense(units=1, activation='sigmoid'))
    model_R.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])
    return model_R

In [49]:
# Train the model
model_R = create_RNN_model()
history = model_R.fit(X_initial_3, y_initial, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 1s 7ms/step - loss: 0.5530 - mse: 0.1827 - val_loss: 0.4370 - val_mse: 0.1269
Epoch 2/50
30/30 [==============================] - 0s 2ms/step - loss: 0.3702 - mse: 0.1004 - val_loss: 0.2928 - val_mse: 0.0668
Epoch 3/50
30/30 [==============================] - 0s 2ms/step - loss: 0.2564 - mse: 0.0564 - val_loss: 0.2026 - val_mse: 0.0362
Epoch 4/50
30/30 [==============================] - 0s 2ms/step - loss: 0.1860 - mse: 0.0340 - val_loss: 0.1450 - val_mse: 0.0205
Epoch 5/50
30/30 [==============================] - 0s 2ms/step - loss: 0.1412 - mse: 0.0223 - val_loss: 0.1066 - val_mse: 0.0121
Epoch 6/50
30/30 [==============================] - 0s 2ms/step - loss: 0.1071 - mse: 0.0144 - val_loss: 0.0807 - val_mse: 0.0075
Epoch 7/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0824 - mse: 0.0094 - val_loss: 0.0628 - val_mse: 0.0048
Epoch 8/50
30/30 [==============================] - 0s 2ms/step - loss: 0.0668 - mse: 0.00

In [50]:
# Evaluate the model on the test set
y_pre=model_R.predict(X_test_3)
y_pred = (y_pre > 0.5).astype(int)

2015/2015 [==============================] - 1s 575us/step


In [51]:
computeMetric(y_true,y_pred)

acc: 0.9991779122072282
balanced acc: 0.5
precision: 0.0
recall: 0.0
cm: [[64417     0]
 [   53     0]]
f1: 0.0
MCC: 0.0
Kappa: 0.0


0.0

## AL
### Baseline

In [ ]:
"""
Training the ActiveLearner
"""
# initialize ActiveLearner
al_learn(model_b,"Random sampling",X_re,y_re,X_test)

In [ ]:
# the final accuracy score
y_pre=model_b.predict(X_test)
y_pred = (y_pre > 0.5).astype(int)
computeMetric(y_true,y_pred)

### RNN

In [ ]:
"""
Training the ActiveLearner
"""
# initialize ActiveLearner
al_learn(model_R,"Random sampling",X_re_3,y_re,X_test_3)

## random sampling

In [ ]:
sampling=random_sampling

In [ ]:
metric04 = al_learn(model_b,sampling,X_initial,y_initial,X_re,y_re)

## Uncertainty

In [ ]:
sampling=uncertainty_sampling

In [ ]:
metric14 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

## entropy

In [ ]:
sampling=entropy_sampling

In [ ]:
metric24 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

## margin

In [ ]:
sampling=margin_sampling

In [ ]:
metric34 = al_learn(clf4,sampling,X_initial,y_initial,X_re,y_re)

# visual


In [ ]:
#save
def con_format(data):
    metrics_arr=np.transpose(metrics)# array T
    df = pd.DataFrame(metrics_arr, columns=['Random_Knn', 'Random_Lighgbm',"Random_GradientBoosting","Random_Catboost",
                                   'Uncertainty_Knn', 'Uncertainty_Lighgbm',"Uncertainty_GradientBoosting","Uncertainty_Catboost",
                                   'Entropy_Knn', 'Entropy_Lighgbm',"Entropy_GradientBoosting","Entropy_Catboost",
                                    'Margin_Knn', 'Margin_Lighgbm',"Margin_GradientBoosting","Margin_Catboost",
                                   ])
    return df
    

In [ ]:
metrics1= list(zip(metric01.iloc[:,1].values,metric02.iloc[:,1].values,metric03.iloc[:,1].values,metric04.iloc[:,1].values,
        metric11.iloc[:,1].values,metric12.iloc[:,1].values,metric13.iloc[:,1].values,metric14.iloc[:,1].values,
        metric21.iloc[:,1].values,metric22.iloc[:,1].values,metric23.iloc[:,1].values,metric24.iloc[:,1].values,
        metric31.iloc[:,1].values,metric32.iloc[:,1].values,metric33.iloc[:,1].values,metric34.iloc[:,1].values))


In [ ]:
df=con_format(metrics)
df

In [ ]:
# df.to_csv("./result/random_f1_3901890.csv")